# [TR-005] Non-relativistic K-matrix

<!-- cspell:ignore Cayley -->

This report investigates how to implement non-relativistic $K$-matrix dynamics with {doc}`SymPy <sympy:index>`. The non-relativistic case is simplest and allows us to check whether the case $n_R=1, n=1$ (single resonance, single channel) reduces to a non-relativistic Breit-Wigner function.[^1] We followed the physics as described by {pdg-review}`Resonances` and {cite}`chungPartialWaveAnalysis1995,petersPartialWaveAnalysis2004,meyerMatrixTutorial2008`.

[^1]: Of course, there is no need to work with matrices in this $1 \times 1$ case. To keeps things general, however, we keep using matrices.

The challenge is to generate a correct parametrization for an arbitrary **number of coupled channels $n$** and an arbitrary **number of resonances $n_R$**. Our approach is to construct an $n \times n$ {class}`sympy.Matrix <sympy.matrices.dense.MutableDenseMatrix>` with {class}`~sympy.core.symbol.Symbol`s as its elements. We then use substitute these {class}`~sympy.core.symbol.Symbol`s with certain parametrizations using {meth}`~sympy.core.basic.Basic.subs`. In order to generate symbols for $n_R$ resonances and $n$ channels, we use {doc}`indexed symbols <modules/tensor/indexed>`.

This approach is less elegant and (theoretically) slower than using {class}`~sympy.matrices.expressions.MatrixSymbol`s. That approach is explored in {doc}`/report/007`.

In [ ]:
%matplotlib widget
import os
import warnings
from typing import Union

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import symplot
import sympy as sp
from IPython.display import Image, Math
from ipywidgets import widgets as ipywidgets
from matplotlib import cm
from mpl_interactions.controller import Controls

warnings.filterwarnings("ignore")
STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

## Procedure

It would be nice to use a {class}`~sympy.core.symbol.Symbol` to represent the number of channels $n$ and specify its value later:

In [ ]:
n_channels = sp.Symbol("n", integer=True, positive=True)

Unfortunately, this does not work well in the {class}`Matrix <sympy.matrices.dense.MutableDenseMatrix>` class. We therefore set variables $n$ to a specific {obj}`int` value and define some other {class}`~sympy.core.symbol.Symbol`s for the rest of the implementation.[^2] The value we choose in this example is `n=1`, because we want to see if this reproduces a non-relativistic Breit-Wigner function.

[^2]: We use {class}`~sympy.core.symbol.Symbol`s as indices, because that renders more nicely.

In [ ]:
n_channels = 1
i, j, R, n_resonances = sp.symbols(
    "i j R n_R", integer=True, negative=False
)
m = sp.Symbol("m", real=True)
M = sp.IndexedBase("m", shape=(n_resonances,))
Gamma = sp.IndexedBase("Gamma", shape=(n_resonances,))
gamma = sp.IndexedBase("gamma", shape=(n_resonances, n_channels))

The $\boldsymbol{T}$-matrix (which expresses the transition amplitudes in which we are interested) can be factorized in terms of the $\boldsymbol{K}$-matrix as follows:

$$
\boldsymbol{T} = \boldsymbol{K}(\boldsymbol{I} - i\boldsymbol{K})^{-1}
$$ (T-matrix)

This expression comes from a [Cayley transformation](https://en.wikipedia.org/wiki/Cayley_transform) on the $\boldsymbol{S}$-matrix, which ensures that the $\boldsymbol{K}$-matrix is real. The challenge is now to choose a correct parametrization. There are several choices, but a common one is the following summation over the resonances $R$:

In [ ]:
def Kij_non_relativistic(
    m: sp.Symbol,
    M: sp.IndexedBase,
    Gamma: sp.IndexedBase,
    gamma: sp.IndexedBase,
    i: int,
    j: int,
    n_resonances: Union[int, sp.Symbol],
) -> sp.Expr:
    g_i = gamma[R, i] * sp.sqrt(M[R] * Gamma[R])
    g_j = gamma[R, j] * sp.sqrt(M[R] * Gamma[R])
    parametrization = (g_i * g_j) / (M[R] ** 2 - m ** 2)
    return sp.Sum(parametrization, (R, 0, n_resonances - 1))

In [ ]:
n_R = sp.Symbol("n_R")
kij = Kij_non_relativistic(m, M, Gamma, gamma, i, j, n_R)
Math("K_{ij} = " + f"{sp.latex(kij)}")

Here, some literature uses residue constants (see {cite}`chungPartialWaveAnalysis1995`, Eq. (82)):

$$
g_{R,i}=\gamma_{R,i}\sqrt{m_R\Gamma_R}
$$ (residue-constant-g)

but to simplify the implementation, we directly plug in Eq. {eq}`residue-constant-g`.

We now define the $\boldsymbol{K}$-matrix in terms of a {class}`Matrix <sympy.matrices.dense.MutableDenseMatrix>` with {class}`~sympy.tensor.indexed.IndexedBase` instances as elements that can serve as {class}`~sympy.core.symbol.Symbol`s. These {class}`~sympy.core.symbol.Symbol`s will be substituted with the parametrization later. We could of course have inserted the parametrization directly, but this slows down matrix multiplication in the following steps.

In [ ]:
K_symbol = sp.IndexedBase("K", shape=(n_channels, n_channels))
K = sp.Matrix(
    [
        [K_symbol[i, j] for j in range(n_channels)]
        for i in range(n_channels)
    ]
)
display(K_symbol[i, j], K)

The $\boldsymbol{T}$-matrix can now be computed from Eq. {eq}`T-matrix`:

In [ ]:
T = K * (sp.eye(n_channels) - sp.I * K).inv()
T

Next, we need to substitute the elements $K_{i,j}$ with the parametrization we defined above:

In [ ]:
T_subs = T.subs(
    {
        K[i, j]: Kij_non_relativistic(
            m, M, Gamma, gamma, i, j, n_resonances
        )
        for i in range(n_channels)
        for j in range(n_channels)
    }
)
T_subs

:::{warning}

It is important to perform {meth}`~sympy.core.basic.Basic.doit` _after_ {meth}`~sympy.core.basic.Basic.subs`, otherwise the {class}`~sympy.concrete.summations.Sum` cannot be evaluated and there will be no warning of a failed substitution.

:::

Now indeed, when taking $n_R=1$, the resulting element from the $\boldsymbol{T}$-matrix looks like a non-relativistic Breit-Wigner function!

In [ ]:
n_resonances_val = 1
rel_bw = T_subs[0, 0].subs(n_resonances, n_resonances_val).doit()
if n_resonances_val == 1 or n == 2:
    rel_bw = rel_bw.simplify()
rel_bw

## Generalization

The above procedure has been condensed into a function that can handle an arbitrary number of resonances and an arbitrary number of channels.

In [ ]:
def create_symbol_matrix(name: str, n: int) -> sp.Matrix:
    symbol = sp.IndexedBase("K", shape=(n, n))
    return sp.Matrix(
        [[symbol[i, j] for j in range(n)] for i in range(n)]
    )


def k_matrix(n_resonances: int, n_channels: int) -> sp.Matrix:
    # Define symbols
    m = sp.Symbol("m", real=True)
    M = sp.IndexedBase("m", shape=(n_resonances,))
    Gamma = sp.IndexedBase("Gamma", shape=(n_resonances,))
    gamma = sp.IndexedBase("gamma", shape=(n_resonances, n_channels))
    # Define K-matrix and T-matrix
    K = create_symbol_matrix("K", n_channels)
    T = K * (sp.eye(n_channels) - sp.I * K).inv()
    # Substitute elements
    return T.subs(
        {
            K[i, j]: Kij_non_relativistic(
                m, M, Gamma, gamma, i, j, n_resonances
            )
            for i in range(n_channels)
            for j in range(n_channels)
        }
    )

Single channel, single resonance:

In [ ]:
k_matrix(n_resonances=1, n_channels=1)[0, 0].doit().simplify()

Single channel, $n_R$ resonances

In [ ]:
k_matrix(n_resonances=sp.Symbol("n_R"), n_channels=1)[0, 0]

Two channels, one resonance ('non-relativistic Flatté'):

In [ ]:
k_matrix(n_resonances=1, n_channels=2)[0, 0].doit().simplify()

Two channels, $n_R$ resonances:

In [ ]:
expr = k_matrix(n_resonances=sp.Symbol("n_R"), n_channels=2)[0, 0]
Math(sp.multiline_latex("", expr))

## Visualization

Now, let's use {mod}`symplot` to visualize the single channel $\boldsymbol{K}$-matrix for arbitrary $n_R$.

:::{margin}

{doc}`/report/008` explains the need for {func}`symplot.substitute_indexed_symbols`.

:::

In [ ]:
def plot_k_matrix(
    n_channels: int,
    n_resonances: int,
    title: str = "",
) -> None:
    # Convert to Symbol: symplot cannot handle IndexedBase
    i = sp.Symbol("i", integer=True, negative=False)
    expr = k_matrix(n_resonances, n_channels)[i, i].doit()
    expr = symplot.substitute_indexed_symbols(expr)
    np_expr, sliders = symplot.prepare_sliders(expr, plot_symbol=m)
    symbol_to_arg = {
        symbol: arg for arg, symbol in sliders._arg_to_symbol.items()
    }

    # Set plot domain
    x_min, x_max = 1e-3, 3
    y_min, y_max = -0.5, +0.5
    z_min, z_max = -2, +2

    plot_domain = np.linspace(x_min, x_max, num=500)
    x_values = np.linspace(x_min, x_max, num=160)
    y_values = np.linspace(y_min, y_max, num=80)
    X, Y = np.meshgrid(x_values, y_values)
    plot_domain_complex = X + Y * 1j

    z_cut_min = 0.75 * z_min
    z_cut_max = 0.75 * z_max
    cut_off_min = np.vectorize(
        lambda z: z if z > z_cut_min else z_cut_min
    )
    cut_off_max = np.vectorize(
        lambda z: z if z < z_cut_max else z_cut_max
    )

    # Set slider values and ranges
    m0_values = np.linspace(x_min, x_max, num=n_resonances + 2)
    m0_values = m0_values[1:-1]

    def set_default_values():
        sliders.set_ranges({"i": (0, n_resonances - 1)})
        for R in range(n_resonances):
            # ranges
            sliders.set_ranges({f"m{R}": (0, 3, 100)})
            sliders.set_ranges({f"Gamma{R}": (-1, 1, 100)})
            for i in range(n_channels):
                sliders.set_ranges(
                    {fR"\gamma_{{{R},{i}}}": (0, 2, 100)}
                )
            # values
            sliders.set_values({f"m{R}": m0_values[R]})
            sliders.set_values({f"Gamma{R}": (R + 1) * 0.1})
            for i in range(n_channels):
                sliders.set_values(
                    {fR"\gamma_{{{R},{i}}}": 1 - 0.1 * R + 0.1 * i}
                )

    set_default_values()

    # Create interactive plots
    controls = Controls(**sliders)
    nrows = 2  # set to 3 for imag+real
    fig, axes = plt.subplots(
        nrows=nrows,
        figsize=(8, nrows * 3.0),
        sharex=True,
        tight_layout=True,
    )
    if not title:
        title = (
            fR"${n_channels} \times {n_channels}$ $K$-matrix"
            f" with {n_resonances} resonances"
        )
    fig.suptitle(title)

    # 2D plot
    axes[0].set_ylabel("$|T|^{2}$")

    def plot(channel: int):
        def wrapped(*args, **kwargs) -> sp.Expr:
            kwargs["i"] = channel
            return np.abs(np_expr(*args, **kwargs)) ** 2

        return wrapped

    for i in range(n_channels):
        iplt.plot(
            plot_domain,
            plot(i),
            ax=axes[0],
            controls=controls,
            ylim="auto",
            label=f"channel {i}",
        )
    if n_channels > 1:
        axes[0].legend(loc="upper right")
    mass_line_style = dict(
        c="red",
        alpha=0.3,
    )
    for name in controls.params:
        if not name.startswith("m"):
            continue
        iplt.axvline(controls[name], ax=axes[0], **mass_line_style)

    # 3D plot
    def plot3(**kwargs):
        Z = np_expr(plot_domain_complex, **kwargs)
        Z_imag = cut_off_min(cut_off_max(Z.imag))
        for ax in axes[1:]:
            ax.clear()
        axes[-1].pcolormesh(X, Y, Z_imag, cmap=cm.coolwarm)
        if kwargs["i"] == 1:
            axes[-1].set_title("Im $T$")
        else:
            axes[-1].set_title(f"Im $T$, channel {i}")
        if len(axes) == 3:
            Z_real = cut_off_min(cut_off_max(Z.real))
            axes[-2].pcolormesh(X, Y, Z_real, cmap=cm.coolwarm)
            axes[-2].set_title("Re $T$")
        for ax in axes[1:]:
            ax.axhline(0, linewidth=0.5, c="black", linestyle="dotted")
            for R in range(n_resonances):
                mass = kwargs[f"m{R}"]
                ax.axvline(mass, **mass_line_style)
            ax.set_ylabel("Im $m$")
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_facecolor("white")
        for R in range(n_resonances):
            mass = kwargs[f"m{R}"]
            axes[-1].text(
                x=mass + (x_max - x_min) * 0.008,
                y=0.95 * y_min,
                s=f"$m_{R}$",
                c="red",
            )
        axes[-1].set_xlabel("Re $m$")
        fig.canvas.draw_idle()

    # Create GUI
    sliders_copy = dict(sliders)
    h_boxes = []
    for R in range(n_resonances):
        buttons = [
            sliders_copy.pop(f"m{R}"),
            sliders_copy.pop(f"Gamma{R}"),
        ]
        if n_channels == 1:
            dummy_name = symbol_to_arg[fR"\gamma_{{{R},0}}"]
            buttons.append(sliders_copy.pop(dummy_name))
        h_box = ipywidgets.HBox(buttons)
        h_boxes.append(h_box)
    remaining_sliders = sorted(
        sliders_copy.values(), key=lambda s: s.description
    )
    ui = ipywidgets.VBox(h_boxes + remaining_sliders)
    output = ipywidgets.interactive_output(plot3, controls=sliders)
    display(ui, output)

In [ ]:
plot_k_matrix(n_resonances=3, n_channels=1)

In [ ]:
if STATIC_WEB_PAGE:
    output_file = "005-K-matrix-n1-r3.png"
    plt.savefig(output_file, dpi=150)
    display(Image(output_file))

In [ ]:
plot_k_matrix(n_resonances=2, n_channels=2)

In [ ]:
if STATIC_WEB_PAGE:
    output_file = "005-K-matrix-n2-r2-00.png"
    plt.savefig(output_file, dpi=150)
    display(Image(output_file))